In [1]:
import pandas as pd
import numpy as np
import networkx as nx
from scipy.io import mmread
import time

In [2]:
from collections import defaultdict
import random

def async_lpa(G, max_iter=100):
    labels = {node: i for i, node in enumerate(G.nodes())}
    
    for _ in range(max_iter):
        changed = False
        nodes = list(G.nodes())
        random.shuffle(nodes)  # 비동기식의 핵심: 무작위 순서
        
        for node in nodes:
            if not G[node]:  # 이웃 없는 노드 스킵
                continue
                
            # 현재 그래프의 실시간 레이블 사용 (비동기식 핵심)
            neighbor_labels = [labels[nbr] for nbr in G[node]]
            if not neighbor_labels:  # 이웃 레이블 없을 경우
                continue
                
            # 레이블 통계 계산
            label_counts = defaultdict(int)
            for lbl in neighbor_labels:
                label_counts[lbl] += 1
                
            max_count = max(label_counts.values())
            candidates = [lbl for lbl, cnt in label_counts.items() 
                         if cnt == max_count]
            
            # 변경 여부 판단
            new_label = random.choice(candidates) if candidates else labels[node]
            if new_label != labels[node]:
                labels[node] = new_label  # 즉시 업데이트 (비동기식)
                changed = True
                
        if not changed:
            break
    
    # 커뮤니티 구성
    comm_dict = defaultdict(list)
    for node, label in labels.items():
        comm_dict[label].append(node)
    return [sorted(nodes) for nodes in comm_dict.values()]


In [3]:
from collections import defaultdict
import random

def sync_lpa(G, max_iter=100):
    labels = {node: i for i, node in enumerate(G.nodes())}
    
    for _ in range(max_iter):
        changed = False
        new_labels = {}  # 동기식의 핵심: 새로운 레이블을 임시 저장
        
        # 모든 노드에 대해 새로운 레이블 계산 (기존 레이블 기준)
        for node in G.nodes():
            if not G[node]:  # 이웃 없는 노드는 기존 레이블 유지
                new_labels[node] = labels[node]
                continue
                
            # 현재 iteration 시작 시점의 레이블 사용 (동기식 핵심)
            neighbor_labels = [labels[nbr] for nbr in G[node]]
            if not neighbor_labels:
                new_labels[node] = labels[node]
                continue
                
            # 레이블 통계 계산
            label_counts = defaultdict(int)
            for lbl in neighbor_labels:
                label_counts[lbl] += 1
                
            max_count = max(label_counts.values())
            candidates = [lbl for lbl, cnt in label_counts.items() 
                         if cnt == max_count]
            
            # 새로운 레이블 결정
            new_label = random.choice(candidates) if candidates else labels[node]
            new_labels[node] = new_label
            
            if new_label != labels[node]:
                changed = True
        
        # 모든 노드의 레이블을 동시에 업데이트 (동기식 핵심)
        labels = new_labels
        
        if not changed:
            break
    
    # 커뮤니티 구성
    comm_dict = defaultdict(list)
    for node, label in labels.items():
        comm_dict[label].append(node)
    return [sorted(nodes) for nodes in comm_dict.values()]


In [4]:
# 최대 독립집합 동기 병령 lpa

from collections import defaultdict
import random
import networkx as nx

def find_maximal_independent_set(G, nodes):
    """탐욕적 방법으로 최대 독립집합 찾기"""
    independent_set = set()
    remaining_nodes = set(nodes)
    
    while remaining_nodes:
        # 남은 노드 중 차수가 가장 낮은 노드 선택 (탐욕적 전략)
        node = min(remaining_nodes, key=lambda n: len([nbr for nbr in G[n] if nbr in remaining_nodes]))
        
        # 독립집합에 추가
        independent_set.add(node)
        remaining_nodes.remove(node)
        
        # 이웃 노드들을 남은 노드에서 제거
        neighbors_to_remove = set(G[node]) & remaining_nodes
        remaining_nodes -= neighbors_to_remove
    
    return independent_set

def sync_lpa_with_mis(G, max_iter=100):
    labels = {node: i for i, node in enumerate(G.nodes())}
    iteration_count = 0
    all_nodes = list(G.nodes())
    
    while iteration_count < max_iter:
        changed = False
        remaining_nodes = set(all_nodes)  # 매 라운드마다 모든 노드로 초기화
        
        # 한 라운드: 모든 노드가 최소 1번씩 업데이트될 때까지
        while remaining_nodes:
            new_labels = labels.copy()
            
            # 남은 노드들에서 독립집합 찾기
            independent_set = find_maximal_independent_set(G, list(remaining_nodes))
            
            # 독립집합의 노드들 업데이트
            for node in independent_set:
                if not G[node]:
                    continue
                    
                neighbor_labels = [labels[nbr] for nbr in G[node]]
                if not neighbor_labels:
                    continue
                    
                label_counts = defaultdict(int)
                for lbl in neighbor_labels:
                    label_counts[lbl] += 1
                    
                max_count = max(label_counts.values())
                candidates = [lbl for lbl, cnt in label_counts.items() 
                             if cnt == max_count]
                
                new_label = random.choice(candidates) if candidates else labels[node]
                new_labels[node] = new_label
                
                if new_label != labels[node]:
                    changed = True
            
            # 업데이트된 노드들을 남은 노드에서 제거
            remaining_nodes -= set(independent_set)
            labels = new_labels
        
        iteration_count += 1
        
        # 변화가 없으면 수렴으로 간주하고 종료
        if not changed:
            break
    
    # 커뮤니티 구성
    comm_dict = defaultdict(list)
    for node, label in labels.items():
        comm_dict[label].append(node)
    return [sorted(nodes) for nodes in comm_dict.values()]


In [5]:
from sklearn.metrics import normalized_mutual_info_score

def calculate_nmi(true_labels, graph, communities):
    """
    true_labels : list of int, 길이 == 노드 수
    graph       : networkx Graph 또는 __len__이 정의된 객체
    communities : List of List or Set, 각 서브리스트/서브셋이 하나의 커뮤니티를 구성하는 노드 ID들
    """
    # pred_labels 초기화: 노드 수만큼 0으로 채운 리스트 생성
    pred_labels = [0] * len(graph)

    # 커뮤니티별 인덱스를 pred_labels에 할당
    for i, com in enumerate(communities):
        for node in com:
            pred_labels[node] = i

    # NMI 계산 및 반환
    return normalized_mutual_info_score(true_labels, pred_labels)


In [6]:
graph = nx.karate_club_graph()
true_labels_karate = []
for node in graph.nodes:
    label = graph.nodes[node]['club']
    true_labels_karate.append(1 if label == 'Officer' else 0)

In [7]:
nmi=[]
elapsedtime=[]
for _ in range(100):
    start_ms = int(round(time.time() * 1000))
    com = async_lpa(graph)
    com
    end_ms = int(round(time.time() * 1000))
    nmi.append( calculate_nmi(true_labels_karate, graph, com ) )
    elapsedtime.append( end_ms - start_ms )
print( "karate    async LPA    mean : ", np.mean(nmi), " std : " , np.var(nmi)**0.5, "time : ", np.mean(elapsedtime))

nmi=[]
elapsedtime=[]
for _ in range(100):
    start_ms = int(round(time.time() * 1000))
    com = sync_lpa(graph)
    com
    end_ms = int(round(time.time() * 1000))
    nmi.append( calculate_nmi(true_labels_karate, graph, com ) )
    elapsedtime.append( end_ms - start_ms )
print( "karate    sync LPA    mean : ", np.mean(nmi), " std : " , np.var(nmi)**0.5, "time : ", np.mean(elapsedtime))

nmi=[]
elapsedtime=[]
for _ in range(100):
    start_ms = int(round(time.time() * 1000))
    com = sync_lpa_with_mis(graph)
    com
    end_ms = int(round(time.time() * 1000))
    nmi.append( calculate_nmi(true_labels_karate, graph, com ) )
    elapsedtime.append( end_ms - start_ms )
print( "karate    sync_mis LPA    mean : ", np.mean(nmi), " std : " , np.var(nmi)**0.5, "time : ", np.mean(elapsedtime))

karate    async LPA    mean :  0.6064299525704413  std :  0.19505698941582847 time :  0.46
karate    sync LPA    mean :  0.4854464036300077  std :  0.20808481189402434 time :  19.54
karate    sync_mis LPA    mean :  0.6592472852084993  std :  0.1252786585038043 time :  8.78


In [8]:
#돌고래
matrix = mmread("./soc-dolphins/soc-dolphins.mtx")
# scipy 희소 행렬을 NetworkX 그래프로 변환

dolphin_graph = nx.from_scipy_sparse_array(matrix)

true_labels_dolphins = [
    0,0,0,0,0,0,0,0,0,0,
    0,1,1,0,0,0,0,0,1,0,
    0,0,0,0,0,0,0,0,0,0,
    1,0,0,0,0,0,0,0,0,0,
    1,1,1,1,1,1,1,1,1,1,
    1,1,1,1,1,1,1,1,1,1,
    1,1
]

In [9]:
nmi=[]
elapsedtime=[]
for _ in range(100):
    start_ms = int(round(time.time() * 1000))
    com = async_lpa(dolphin_graph)
    com
    end_ms = int(round(time.time() * 1000))
    nmi.append( calculate_nmi(true_labels_dolphins, dolphin_graph, com ) )
    elapsedtime.append( end_ms - start_ms )
print( "dolphins    async LPA    mean : ", np.mean(nmi), " std : " , np.var(nmi)**0.5, "time : ", np.mean(elapsedtime))

nmi=[]
elapsedtime=[]
for _ in range(100):
    start_ms = int(round(time.time() * 1000))
    com = sync_lpa(dolphin_graph)
    com
    end_ms = int(round(time.time() * 1000))
    nmi.append( calculate_nmi(true_labels_dolphins, dolphin_graph, com ) )
    elapsedtime.append( end_ms - start_ms )
print( "dolphins    sync LPA    mean : ", np.mean(nmi), " std : " , np.var(nmi)**0.5, "time : ", np.mean(elapsedtime))

nmi=[]
elapsedtime=[]
for _ in range(100):
    start_ms = int(round(time.time() * 1000))
    com = sync_lpa_with_mis(dolphin_graph)
    com
    end_ms = int(round(time.time() * 1000))
    nmi.append( calculate_nmi(true_labels_dolphins, dolphin_graph, com ) )
    elapsedtime.append( end_ms - start_ms )
print( "dolphins    sync_mis LPA    mean : ", np.mean(nmi), " std : " , np.var(nmi)**0.5, "time : ", np.mean(elapsedtime))

dolphins    async LPA    mean :  0.024975094530546676  std :  0.015478228741230777 time :  7.65
dolphins    sync LPA    mean :  0.02357895918678621  std :  0.011109600890087674 time :  33.63
dolphins    sync_mis LPA    mean :  0.03542398849612667  std :  0.025133252906735896 time :  47.6


In [10]:
from torch_geometric.datasets import Planetoid
from torch_geometric.utils import to_networkx

dataset = Planetoid(root='/tmp/Cora', name='Cora')
data = dataset[0]
cora_graph = to_networkx(data)
true_labels_cora = data.y

In [11]:
nmi=[]
elapsedtime=[]
for _ in range(100):
    start_ms = int(round(time.time() * 1000))
    com = async_lpa(cora_graph)
    com
    end_ms = int(round(time.time() * 1000))
    nmi.append( calculate_nmi(true_labels_cora, cora_graph, com ) )
    elapsedtime.append( end_ms - start_ms )
print( "cora    async LPA    mean : ", np.mean(nmi), " std : " , np.var(nmi)**0.5, "time : ", np.mean(elapsedtime))

nmi=[]
elapsedtime=[]
for _ in range(100):
    start_ms = int(round(time.time() * 1000))
    com = sync_lpa(cora_graph)
    com
    end_ms = int(round(time.time() * 1000))
    nmi.append( calculate_nmi(true_labels_cora, cora_graph, com ) )
    elapsedtime.append( end_ms - start_ms )
print( "cora    sync LPA    mean : ", np.mean(nmi), " std : " , np.var(nmi)**0.5, "time : ", np.mean(elapsedtime))

nmi=[]
elapsedtime=[]
for _ in range(10):
    start_ms = int(round(time.time() * 1000))
    com = sync_lpa_with_mis(cora_graph)
    com
    end_ms = int(round(time.time() * 1000))
    nmi.append( calculate_nmi(true_labels_cora, cora_graph, com ) )
    elapsedtime.append( end_ms - start_ms )
print( "cora    sync_mis LPA    mean : ", np.mean(nmi), " std : " , np.var(nmi)**0.5, "time : ", np.mean(elapsedtime))

cora    async LPA    mean :  0.4287923552520222  std :  0.0049359924564645365 time :  2724.36
cora    sync LPA    mean :  0.41274767686855846  std :  0.0064953059922281285 time :  2607.29
cora    sync_mis LPA    mean :  0.427814400039295  std :  0.0038574877442530218 time :  613777.6


In [12]:
for _ in range(10):
    start_ms = int(round(time.time() * 1000))
    com = sync_lpa_with_mis(cora_graph)
    com
    end_ms = int(round(time.time() * 1000))
    nmi.append( calculate_nmi(true_labels_cora, cora_graph, com ) )
    elapsedtime.append( end_ms - start_ms )
print( "cora    sync_mis LPA    mean : ", np.mean(nmi), " std : " , np.var(nmi)**0.5, "time : ", np.mean(elapsedtime))

cora    sync_mis LPA    mean :  0.42782666459831775  std :  0.004237018498066012 time :  613359.0


In [13]:
for _ in range(10):
    start_ms = int(round(time.time() * 1000))
    com = sync_lpa_with_mis(cora_graph)
    com
    end_ms = int(round(time.time() * 1000))
    nmi.append( calculate_nmi(true_labels_cora, cora_graph, com ) )
    elapsedtime.append( end_ms - start_ms )
print( "cora    sync_mis LPA    mean : ", np.mean(nmi), " std : " , np.var(nmi)**0.5, "time : ", np.mean(elapsedtime))

cora    sync_mis LPA    mean :  0.42860557459540655  std :  0.00411965599503738 time :  612962.5


In [14]:
for _ in range(10):
    start_ms = int(round(time.time() * 1000))
    com = sync_lpa_with_mis(cora_graph)
    com
    end_ms = int(round(time.time() * 1000))
    nmi.append( calculate_nmi(true_labels_cora, cora_graph, com ) )
    elapsedtime.append( end_ms - start_ms )
print( "cora    sync_mis LPA    mean : ", np.mean(nmi), " std : " , np.var(nmi)**0.5, "time : ", np.mean(elapsedtime))

cora    sync_mis LPA    mean :  0.4287261822064224  std :  0.004066486199529049 time :  612758.7


In [15]:
for _ in range(10):
    start_ms = int(round(time.time() * 1000))
    com = sync_lpa_with_mis(cora_graph)
    com
    end_ms = int(round(time.time() * 1000))
    nmi.append( calculate_nmi(true_labels_cora, cora_graph, com ) )
    elapsedtime.append( end_ms - start_ms )
print( "cora    sync_mis LPA    mean : ", np.mean(nmi), " std : " , np.var(nmi)**0.5, "time : ", np.mean(elapsedtime))

cora    sync_mis LPA    mean :  0.42929331520798625  std :  0.004154880878382615 time :  612581.28


In [16]:
for _ in range(10):
    start_ms = int(round(time.time() * 1000))
    com = sync_lpa_with_mis(cora_graph)
    com
    end_ms = int(round(time.time() * 1000))
    nmi.append( calculate_nmi(true_labels_cora, cora_graph, com ) )
    elapsedtime.append( end_ms - start_ms )
print( "cora    sync_mis LPA    mean : ", np.mean(nmi), " std : " , np.var(nmi)**0.5, "time : ", np.mean(elapsedtime))

cora    sync_mis LPA    mean :  0.4293420243945339  std :  0.004127571721192989 time :  612759.25


In [17]:
for _ in range(10):
    start_ms = int(round(time.time() * 1000))
    com = sync_lpa_with_mis(cora_graph)
    com
    end_ms = int(round(time.time() * 1000))
    nmi.append( calculate_nmi(true_labels_cora, cora_graph, com ) )
    elapsedtime.append( end_ms - start_ms )
print( "cora    sync_mis LPA    mean : ", np.mean(nmi), " std : " , np.var(nmi)**0.5, "time : ", np.mean(elapsedtime))

cora    sync_mis LPA    mean :  0.42940803454884335  std :  0.004068424068396476 time :  615346.9285714285


In [18]:
for _ in range(10):
    start_ms = int(round(time.time() * 1000))
    com = sync_lpa_with_mis(cora_graph)
    com
    end_ms = int(round(time.time() * 1000))
    nmi.append( calculate_nmi(true_labels_cora, cora_graph, com ) )
    elapsedtime.append( end_ms - start_ms )
print( "cora    sync_mis LPA    mean : ", np.mean(nmi), " std : " , np.var(nmi)**0.5, "time : ", np.mean(elapsedtime))

cora    sync_mis LPA    mean :  0.4291765128539712  std :  0.003946268055472523 time :  616769.575


In [19]:
for _ in range(10):
    start_ms = int(round(time.time() * 1000))
    com = sync_lpa_with_mis(cora_graph)
    com
    end_ms = int(round(time.time() * 1000))
    nmi.append( calculate_nmi(true_labels_cora, cora_graph, com ) )
    elapsedtime.append( end_ms - start_ms )
print( "cora    sync_mis LPA    mean : ", np.mean(nmi), " std : " , np.var(nmi)**0.5, "time : ", np.mean(elapsedtime))

cora    sync_mis LPA    mean :  0.4291168396616611  std :  0.0038296921508026416 time :  616486.3333333334


In [20]:
for _ in range(10):
    start_ms = int(round(time.time() * 1000))
    com = sync_lpa_with_mis(cora_graph)
    com
    end_ms = int(round(time.time() * 1000))
    nmi.append( calculate_nmi(true_labels_cora, cora_graph, com ) )
    elapsedtime.append( end_ms - start_ms )
print( "cora    sync_mis LPA    mean : ", np.mean(nmi), " std : " , np.var(nmi)**0.5, "time : ", np.mean(elapsedtime))

cora    sync_mis LPA    mean :  0.42917784514353763  std :  0.00379867953882354 time :  617697.13


In [21]:
from collections import defaultdict
import random
import networkx as nx
import numpy as np

def optimized_sync_lpa_with_mis(G, max_iter=100):
    """최적화된 동기 병렬 LPA with MIS"""
    
    # 초기화 및 사전 계산
    nodes = list(G.nodes())
    node_to_idx = {node: i for i, node in enumerate(nodes)}
    n_nodes = len(nodes)
    
    # 이웃 정보 사전 계산 (리스트로 저장하여 빠른 접근)
    neighbors = [list(G[node]) for node in nodes]
    neighbor_indices = [[node_to_idx[nbr] for nbr in neighbors[i]] 
                       for i in range(n_nodes)]
    
    # 차수 정보 사전 계산
    degrees = np.array([len(neighbors[i]) for i in range(n_nodes)])
    
    # 라벨 초기화 (numpy 배열 사용)
    labels = np.arange(n_nodes, dtype=np.int32)
    
    # 최대 라벨 수 추정 (메모리 할당 최적화)
    max_labels = min(n_nodes, 1000)  # 실제 사용될 라벨 수는 보통 적음
    
    for iteration in range(max_iter):
        changed = False
        remaining_mask = np.ones(n_nodes, dtype=bool)
        
        while np.any(remaining_mask):
            # 남은 노드들의 인덱스
            remaining_indices = np.where(remaining_mask)[0]
            
            if len(remaining_indices) == 0:
                break
            
            # 독립집합 찾기 (최적화된 버전)
            independent_set = find_optimized_mis(
                remaining_indices, neighbor_indices, remaining_mask, degrees
            )
            
            if not independent_set:
                break
            
            # 라벨 업데이트 (벡터화)
            new_labels = update_labels_vectorized(
                independent_set, neighbor_indices, labels, max_labels
            )
            
            # 변화 감지
            for idx in independent_set:
                if new_labels[idx] != labels[idx]:
                    changed = True
                    labels[idx] = new_labels[idx]
            
            # 처리된 노드들을 마스크에서 제거
            remaining_mask[independent_set] = False
        
        if not changed:
            break
    
    # 커뮤니티 구성 (최적화된 버전)
    return build_communities_optimized(nodes, labels)

def find_optimized_mis(remaining_indices, neighbor_indices, remaining_mask, degrees):
    """최적화된 최대 독립집합 찾기"""
    if len(remaining_indices) == 0:
        return []
    
    independent_set = []
    local_remaining = set(remaining_indices)
    
    # 차수 기준으로 정렬 (한 번만)
    sorted_indices = sorted(remaining_indices, 
                           key=lambda idx: sum(1 for nbr in neighbor_indices[idx] 
                                             if nbr in local_remaining))
    
    for node_idx in sorted_indices:
        if node_idx not in local_remaining:
            continue
            
        independent_set.append(node_idx)
        local_remaining.discard(node_idx)
        
        # 이웃들을 효율적으로 제거
        for nbr_idx in neighbor_indices[node_idx]:
            local_remaining.discard(nbr_idx)
    
    return independent_set

def update_labels_vectorized(independent_set, neighbor_indices, labels, max_labels):
    """벡터화된 라벨 업데이트"""
    new_labels = labels.copy()
    
    for node_idx in independent_set:
        nbr_indices = neighbor_indices[node_idx]
        if not nbr_indices:
            continue
        
        # 이웃 라벨들 수집
        neighbor_labels = labels[nbr_indices]
        
        # 빈도 계산 (numpy 사용)
        unique_labels, counts = np.unique(neighbor_labels, return_counts=True)
        max_count = np.max(counts)
        
        # 최대 빈도를 가진 라벨들 중 랜덤 선택
        candidates = unique_labels[counts == max_count]
        new_labels[node_idx] = np.random.choice(candidates)
    
    return new_labels

def build_communities_optimized(nodes, labels):
    """최적화된 커뮤니티 구성"""
    # numpy의 unique를 활용한 빠른 그룹화
    unique_labels, inverse_indices = np.unique(labels, return_inverse=True)
    
    communities = [[] for _ in range(len(unique_labels))]
    for i, comm_idx in enumerate(inverse_indices):
        communities[comm_idx].append(nodes[i])
    
    return [sorted(comm) for comm in communities if comm]

# 추가 최적화: 작은 그래프용 특별 처리
def fast_sync_lpa_small_graph(G, max_iter=100):
    """작은 그래프를 위한 특별 최적화된 버전"""
    if len(G) < 100:  # 작은 그래프의 경우
        return optimized_sync_lpa_with_mis(G, max_iter)
    
    # 큰 그래프의 경우 추가 최적화 적용
    return optimized_sync_lpa_with_mis(G, max_iter)


In [22]:
optimized_sync_lpa_with_mis(cora_graph)

[[12, 1001, 1318, 2661, 2662],
 [17, 927, 1247, 1301, 1315, 1316, 1679, 2140],
 [44, 1582, 2624, 2701],
 [639, 811, 855, 900, 1662, 1666, 2381, 2398, 2493],
 [71, 206, 2691],
 [85, 2487, 2488],
 [55,
  60,
  76,
  88,
  130,
  162,
  204,
  211,
  272,
  300,
  323,
  325,
  331,
  355,
  356,
  415,
  437,
  438,
  498,
  516,
  593,
  718,
  737,
  787,
  815,
  851,
  966,
  1079,
  1174,
  1218,
  1259,
  1288,
  1309,
  1394,
  1474,
  1487,
  1527,
  1658,
  1667,
  1668,
  1677,
  1696,
  1713,
  1741,
  1843,
  1847,
  1848,
  1882,
  1908,
  1909,
  1983,
  1985,
  2011,
  2012,
  2013,
  2014,
  2015,
  2017,
  2018,
  2019,
  2020,
  2104,
  2116,
  2137,
  2178,
  2261,
  2274,
  2342,
  2467,
  2468,
  2593],
 [54, 104, 401, 864, 1065, 1096, 1633, 2210],
 [118, 255, 554, 685, 1029, 1343, 1690, 2030, 2112, 2165, 2166, 2316],
 [26, 99, 122, 123, 127, 2454, 2455, 2604],
 [135, 137, 2095, 2144, 2329, 2331, 2504],
 [167, 168, 1056, 2437, 2438, 2482],
 [182, 183, 508, 997, 1768,